# 🏭 ImgCraft Chapter Processor

Xử lý manga chapter tuần tự. Upload `.aniga` → Clean page-by-page → Download `.aniga`.

**Bước 1:** Mount Drive → **Bước 2:** Clone repo → **Bước 3:** Install → **Bước 4:** Chạy server

In [ ]:
# ====== Cell 1: Mount Google Drive ======
from google.colab import drive
drive.mount('/content/drive')

import os
WORK_DIR = '/content/drive/MyDrive/Aniga_Work/imgcraft'
os.makedirs(WORK_DIR, exist_ok=True)
os.environ['ANIGA_WORK_DIR'] = WORK_DIR
print(f'✅ Work dir: {WORK_DIR}')

In [ ]:
# ====== Cell 2: Clone repo + Install dependencies ======
    REPO_URL = 'https://github.com/jofix2004/Aniga-Pipeline.git'  # ← Thay URL repo thật
    REPO_DIR = '/content/Aniga-Pipeline/aniga_imgcraft'

if os.path.exists(REPO_DIR):
    %cd {REPO_DIR}
    !git pull
else:
    !git clone {REPO_URL} {REPO_DIR}
    %cd {REPO_DIR}

!pip install -q fastapi uvicorn pillow

# ====== Thêm import cho pipeline ImgCraft thật ======
# !pip install -q diffusers accelerate transformers
# from pipeline_v2 import FluxProcessor

print('✅ Repo + dependencies ready')

In [ ]:
# ====== Cell 3: (Optional) Init FluxProcessor ======
# Bỏ comment khi dùng thật:
#
# import sys
# sys.path.insert(0, '/content/imgcraft_pipeline_v2')
# from pipeline_v2 import FluxProcessor
# 
# flux = FluxProcessor()  # Load model
# 
# def process_page(raw_pil):
#     tiles = flux.crop_4_tiles(flux.center(raw_pil))
#     clean_tiles = flux.process(tiles)
#     return flux.stitch(raw_pil, clean_tiles)
# 
# # Set vào server state
# from imgcraft_server import state
# state.flux_processor = type('P', (), {'process_page': staticmethod(process_page)})()

print('⚠️ FluxProcessor chưa init — chạy ở Demo mode')
print('   Để dùng thật, bỏ comment đoạn code trên')

In [ ]:
# ====== Cell 4: Start server + Tunnel ======
import threading
import subprocess
import time
import re

PORT = 8001

# Start FastAPI trong background thread
def run_server():
    import uvicorn
    from imgcraft_server import app
    uvicorn.run(app, host='0.0.0.0', port=PORT, access_log=False)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(2)
print(f'✅ Server chạy tại port {PORT}')

# ====== Cloudflare Tunnel ======
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared

tunnel_proc = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', f'http://localhost:{PORT}'],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
)

# Đợi URL
tunnel_url = None
for _ in range(30):
    line = tunnel_proc.stderr.readline()
    match = re.search(r'(https://[a-z0-9\-]+\.trycloudflare\.com)', line)
    if match:
        tunnel_url = match.group(1)
        break
    time.sleep(0.5)

if tunnel_url:
    print(f'\n🌐 PUBLIC URL: {tunnel_url}')
    print(f'   Mở link trên để sử dụng ImgCraft Processor')
else:
    print('❌ Không lấy được tunnel URL. Kiểm tra lại!')